#### 1. Load the required packages and dependencies

In [26]:
%pip install argparse
%pip install matplotlib
%pip install tifffile

import argparse as args
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import tifffile as tiff
from pathlib import Path
from sklearn.linear_model import LinearRegression, RANSACRegressor

  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


#### 2. Load the input image and files

In [28]:
# Define paths
image_path = Path("C:/Users/g/Downloads/test.ome 1.tif")   # the OME-TIFF path
csv_path = Path("C:/Users/g/Downloads/target_background_pairs.csv") # the input CSV path
output_dir = Path("C:/Users/g/Downloads/all_corrected_channels")
output_dir.mkdir(exist_ok=True)

# Load the OME-TIFF image
arr = tiff.imread(image_path)  # shape (C, H, W)
print(f"Loaded image with shape: {arr.shape}")

# Load the csv with channel pairs
# Accept either '1;7' single-column rows or two-column CSVs; skip a header row if present.
# Use a regex sep to accept both semicolon and comma separators.
df_pairs = pd.read_csv(csv_path, dtype=str, sep='[;,]', engine='python', comment='#')

pairs = []
# NOTE: CSV channels are often 1-based. Convert to 0-based indices to index numpy arrays.
# If the file was a single column (e.g., each row "1;7"), split the values
if df_pairs.shape[1] == 1:
    for val in df_pairs.iloc[:, 0].astype(str):
        s = val.strip()
        # skip an explicit header row if present
        if not s or s.lower().startswith('target') or s.lower().startswith('background'):
            continue
        parts = [p.strip() for p in s.replace(',', ';').split(';')]
        if len(parts) < 2:
            raise ValueError(f"Could not parse channel pair from value '{val}'")
        try:
            # convert to zero-based indices
            t, b = int(parts[0]) - 1, int(parts[1]) - 1
        except ValueError:
            raise ValueError(f"Could not parse channel pair from value '{val}'")
        pairs.append((t, b))
else:
    # Multiple columns: use the first two columns
    for idx, row in df_pairs.iloc[:, :2].iterrows():
        a, b = str(row.iloc[0]).strip(), str(row.iloc[1]).strip()
        # skip header-like rows
        if not a or not b or a.lower().startswith('target') or b.lower().startswith('background'):
            continue
        try:
            # convert to zero-based indices
            t, bg = int(a) - 1, int(b) - 1
        except ValueError:
            raise ValueError(f"Could not parse channel pair from row {list(row.values)}")
        pairs.append((t, bg))

if len(pairs) == 0:
    raise ValueError("No valid channel pairs found in CSV.")

print("Channel pairs for correction (0-based indices):\n", pairs)

Loaded image with shape: (7, 5376, 4056)
Channel pairs for correction (0-based indices):
 [(0, 6), (1, 6), (2, 6), (3, 6), (4, 6), (5, 6)]


#### 3. Define tile parameters

In [ ]:
tile_size = 1000  # Size of each tile (1000x1000 pixels)
num_tiles = 10    # Maximum number of tiles to sample per channel pair

#### 4. Loop over every channel to select tiles with high signal intensity and minimal background pixels

In [29]:
results = {}  # dictionary to store results for each channel pair

# Loop over each target/background pair from the CSV
for target_chan, bg_chan in pairs:
    print(f"\n🔹 Correcting Channel {target_chan} using background Channel {bg_chan}")

    ch_target = arr[target_chan].astype(np.float32)
    ch_bg = arr[bg_chan].astype(np.float32)
    C, H, W = arr.shape

    # Compute global stats
    mean_intensity = ch_target.mean()
    max_intensity = ch_target.max()
    print(f"Channel {target_chan} - mean: {mean_intensity:.2f}, max: {max_intensity:.2f}")

    # Set adaptive thresholds based on global stats
    mean_thresh = mean_intensity * 1.5   # 50% brighter than average
    max_thresh = max_intensity * 0.3     # top 30% of max
    print(f"Adaptive thresholds → mean > {mean_thresh:.1f}, max > {max_thresh:.1f}")

    # Find bright tiles
    bright_tiles_coords = []
    for y in range(0, H - tile_size, tile_size):
        for x in range(0, W - tile_size, tile_size):
            tile = ch_target[y:y+tile_size, x:x+tile_size]
            if tile.mean() > mean_thresh and tile.max() > max_thresh:
                bright_tiles_coords.append((y, x))

    print(f"Found {len(bright_tiles_coords)} bright tiles.")

    # Select tiles
    if len(bright_tiles_coords) == 0:
        print("No bright tiles found. Skipping this channel.")
        continue
    elif len(bright_tiles_coords) < num_tiles:
        coords = bright_tiles_coords
        print("Fewer bright tiles than requested, using all available.")
    else:
        coords = random.sample(bright_tiles_coords, num_tiles)

    print("Selected bright tile coordinates (y, x):", coords)

    # Save coords for use in the next step
    results[target_chan] = {"background": bg_chan, "coords": coords}



🔹 Correcting Channel 0 using background Channel 6
Channel 0 - mean: 527.59, max: 18880.00
Adaptive thresholds → mean > 791.4, max > 5664.0
Found 6 bright tiles.
Fewer bright tiles than requested, using all available.
Selected bright tile coordinates (y, x): [(2000, 0), (2000, 1000), (2000, 2000), (2000, 3000), (3000, 1000), (3000, 2000)]

🔹 Correcting Channel 1 using background Channel 6
Channel 1 - mean: 849.17, max: 64105.00
Adaptive thresholds → mean > 1273.8, max > 19231.5
Found 6 bright tiles.
Fewer bright tiles than requested, using all available.
Selected bright tile coordinates (y, x): [(2000, 0), (2000, 1000), (2000, 2000), (2000, 3000), (3000, 1000), (3000, 2000)]

🔹 Correcting Channel 2 using background Channel 6
Channel 2 - mean: 295.55, max: 38333.00
Adaptive thresholds → mean > 443.3, max > 11499.9
Found 6 bright tiles.
Fewer bright tiles than requested, using all available.
Selected bright tile coordinates (y, x): [(2000, 0), (2000, 1000), (2000, 2000), (2000, 3000), (3

#### 5. Estimate and apply the background co-efficient

In [30]:
# Estimate correction coefficients and apply correction 
for target_chan, info in results.items():
    bg_chan = info["background"]
    coords = info["coords"]

    print(f"\n Estimating correction for Channel {target_chan} (BG {bg_chan})")

    ch_target = arr[target_chan].astype(np.float32)
    ch_bg = arr[bg_chan].astype(np.float32)

    # Gather pixels from bright tiles
    bg_values = []
    target_values = []
    for (y, x) in coords:
        tile_target = ch_target[y:y+tile_size, x:x+tile_size].flatten()
        tile_bg = ch_bg[y:y+tile_size, x:x+tile_size].flatten()
        bg_values.append(tile_bg)
        target_values.append(tile_target)

    # Combine all tile pixels
    bg_values = np.concatenate(bg_values)
    target_values = np.concatenate(target_values)

    # Linear least-squares fit: target ≈ a * bg + b
    A = np.vstack([bg_values, np.ones_like(bg_values)]).T
    coef, intercept = np.linalg.lstsq(A, target_values, rcond=None)[0]

    # Apply correction
    corrected = ch_target - (coef * ch_bg + intercept)
    corrected = np.clip(corrected, 0, None).astype(np.float32)

    # Save results (store coefficient + intercept)
    results[target_chan].update({"coef": float(coef), "intercept": float(intercept)})

    print(f"Channel {target_chan}: coef = {coef:.4f}, intercept = {intercept:.2f}")

# Summary
print("\n Summary of all correction coefficients:")
for ch, vals in results.items():
    print(f"Channel {ch} (BG {vals['background']}): coef={vals['coef']:.4f}, intercept={vals['intercept']:.2f}")



 Estimating correction for Channel 0 (BG 6)
Channel 0: coef = 0.7472, intercept = -950.01

 Estimating correction for Channel 1 (BG 6)
Channel 1: coef = 1.0932, intercept = -1175.02

 Estimating correction for Channel 2 (BG 6)
Channel 2: coef = 0.3908, intercept = -498.00

 Estimating correction for Channel 3 (BG 6)
Channel 3: coef = 0.4174, intercept = -481.57

 Estimating correction for Channel 4 (BG 6)
Channel 4: coef = 0.0739, intercept = -63.28

 Estimating correction for Channel 5 (BG 6)
Channel 5: coef = 0.4002, intercept = -580.96

 Summary of all correction coefficients:
Channel 0 (BG 6): coef=0.7472, intercept=-950.01
Channel 1 (BG 6): coef=1.0932, intercept=-1175.02
Channel 2 (BG 6): coef=0.3908, intercept=-498.00
Channel 3 (BG 6): coef=0.4174, intercept=-481.57
Channel 4 (BG 6): coef=0.0739, intercept=-63.28
Channel 5 (BG 6): coef=0.4002, intercept=-580.96


#### 6. Save the calculated co-efficients per channel

In [31]:
# Save results to CSV
df_results = pd.DataFrame([
    {"target_channel": ch, **vals} for ch, vals in results.items()
])
df_results.to_csv("C:/Users/g/Downloads/correction_coefficients.csv", index=False)
print("Saved coefficients to correction_coefficients.csv")


Saved coefficients to correction_coefficients.csv


#### 7. Save the corrected output image with the all corrected channels

In [33]:
# Save all corrected channels as a single OME-TIFF
output_path = Path("C:/Users/g/Downloads/7ch_corrected_allchannels.ome.tif")

# Build arr_corrected from original arr and the computed coefficients in `results`.
# If a channel was corrected (present in results), apply correction:
# corrected = ch_target - (coef * ch_bg + intercept)
# Otherwise keep the original channel.
arr_corrected = arr.astype(np.float32).copy()

# Ensure results is available and contains coefficients for target channels
for ch_idx in range(C):
    if ch_idx in results:
        info = results[ch_idx]
        bg_idx = info["background"]
        coef = float(info["coef"])
        intercept = float(info["intercept"])

        ch_target = arr[ch_idx].astype(np.float32)
        ch_bg = arr[bg_idx].astype(np.float32)

        corrected = ch_target - (coef * ch_bg + intercept)
        # Clip to valid uint16 range before converting
        corrected = np.clip(corrected, 0, 65535)
        arr_corrected[ch_idx] = corrected
    else:
        # keep original (already in arr_corrected)
        continue

# Convert to uint16 for saving
arr_corrected = arr_corrected.astype(np.uint16)

tiff.imwrite(
    output_path,
    arr_corrected,
    photometric='minisblack'
)

print(f"\nFull {C}-channel corrected image saved to: {output_path}")



Full 7-channel corrected image saved to: C:\Users\g\Downloads\7ch_corrected_allchannels.ome.tif
